# git管理 TODO（現在使ってない）

In [ ]:
!pip install colab-xterm
%load_ext colabxterm

# Flowchart-Detection


In [ ]:
!apt-cache search python3

In [ ]:
!pip freeze

In [ ]:
# 関連ライブラリのインストール
!pip install cairosvg
!apt install tree
!pip install xmltodict
!pip install albumentations
!pip install ultralytics

In [ ]:
!pwd

/content


In [ ]:
!nvidia-smi

In [ ]:
# g-driveのマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/programs/flow-chart-detection/

ckpt  data  data_yolo  images  output  runs_l  runs_n


In [ ]:
# datasetへのpathを設定
PATH_TO_FCDetection = '/content/drive/MyDrive/programs/flow-chart-detection'

In [ ]:
!tree /content/drive/MyDrive/programs/flow-chart-detection

# 画像の読み込みと中身確認

In [ ]:
# ライブラリのインストール
from PIL import Image
import cairosvg
from io import BytesIO
import cv2
from IPython.display import display
import numpy as np
import os
import glob

In [ ]:
#画像の読み込みと表示
files = glob.glob(os.path.join(PATH_TO_FCDetection, 'data', '*'))
print(f"files: {files}")
for path1 in files:
  if path1.rsplit('.', 1)[1] == 'xml' or path1.rsplit('.', 1)[1] == 'txt':
    continue
  if path1.rsplit('.', 1)[1] == 'svg':
    img = cairosvg.svg2png(url=path1)
    img = Image.open(BytesIO(img))
  else:
    img = Image.open(path1).convert('RGB')
  print("type(img), ", type(img))
  print("img.size, ", img.size)
  # img = cv2.imread(path1)
  display(img)

## アノテーションの読み込み

In [ ]:
import xml.etree.ElementTree as ET
for path1 in files:
  if path1.rsplit('.', 1)[1] != 'xml':
    continue
  tree = ET.parse(path1)
  root = tree.getroot()
  for child in root:
    print(child.tag, child.attrib)

## 画像にアノテーションを描画する

In [ ]:
from PIL import ImageDraw
import xmltodict
# xmlのみ収集
xml_files = []
files = glob.glob(os.path.join(PATH_TO_FCDetection, 'data', '*'))
for path1 in files:
  if path1.rsplit('.', 1)[1] == 'xml':
    xml_files.append(path1)
print('xml_files', xml_files)

# xmlファイルから対応する画像ファイルを読み込む
for xml_path1 in xml_files:
  # get image path... png, jpg, jpeg, ...etc
  # img_path_base = xml_path1.rsplit('.', 1)[0]
  # files = glob.glob(os.path.join(PATH_TO_FCDetection, 'data', img_path_base + '.*'))
  # img_file_name = None
  # for path1 in files:
  #   if path1.rsplit('.', 1)[1] != 'xml':
  #     img_file_name = path1

  with open(xml_path1, 'r') as f:
      xml_string = f.read()
  xml_dict = xmltodict.parse(xml_string)

  colors = {'terminator':"red",
            'process':"green",
            'decision':"#808080",
            'arrow':"#FFA500",
            'text':"blue",
            'start_point':"#800080",
            'data':"grey",
            'connection':"black",
            'end_point':"yellow"}

  anno_dict = xml_dict['annotation']
  # open image
  filename = anno_dict['filename']
  file_path = os.path.join(PATH_TO_FCDetection, 'data', filename)
  print("file_path, ", file_path)
  # if filename.rsplit('.', 1)[1] == 'gif':
  img = Image.open(file_path).convert('RGB')
  # else:
  #   img = Image.open(file_path)
  print("type(img), ", type(img))
  print("img.size, ", img.size)
  # ImageDrawオブジェクトの作成
  draw = ImageDraw.Draw(img)

  for key, value in anno_dict.items():
    if key == 'object':
      for object1 in value:
        print("object1, ", object1)
        xmin = int(object1['bndbox']['xmin'])
        ymin = int(object1['bndbox']['ymin'])
        xmax = int(object1['bndbox']['xmax'])
        ymax = int(object1['bndbox']['ymax'])

        # 矩形の描画
        # (x1, y1)は左上の座標、(x2, y2)は右下の座標
        print("")
        draw.rectangle((xmin, ymin, xmax, ymax), outline=colors[object1['name']],
                       width=3) # fill=colors[object1['name']],

  # 画像の保存
  # img.save('rectangle.png')
  display(img)



# モデルの構築と学習

## importするtorch関係ライブラリ

In [ ]:
import xml.etree.ElementTree as ET
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from torchvision.models.detection import fcos_resnet50_fpn
from torchvision.models.detection.fcos import FCOSHead
import cv2
import numpy as np
from PIL import Image
import xmltodict
import albumentations as A
from albumentations.pytorch import ToTensorV2
import os

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.23 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


## annotationの前処理

In [ ]:
LABELS = {'text':0,
          'arrow':1,
          'connection':2,
          'data':3,
          'decision':4,
          'process':5,
          'terminator':6}


def convert_xml_to_yolo(xml_dir, output_dir, image_dir):
    os.makedirs(output_dir, exist_ok=True)
    xml_files = [f for f in os.listdir(xml_dir) if f.endswith('.xml')]

    for xml_file in xml_files:
        # XMLファイルを読み込み
        tree = ET.parse(os.path.join(xml_dir, xml_file))
        root = tree.getroot()

        # 画像情報を取得
        filename = root.find('filename').text
        img_path = os.path.join(image_dir, filename)
        img_width = int(root.find('size/width').text)
        img_height = int(root.find('size/height').text)

        # 出力先ファイル名
        output_txt_path = os.path.join(output_dir, os.path.splitext(filename)[0] + '.txt')

        # バウンディングボックスを変換
        with open(output_txt_path, 'w') as f:
            for obj in root.findall('object'):
                class_name = obj.find('name').text
                if class_name not in LABELS:
                    continue  # 定義されていないクラスはスキップ
                class_id = LABELS[class_name]
                bndbox = obj.find('bndbox')
                xmin = int(bndbox.find('xmin').text)
                ymin = int(bndbox.find('ymin').text)
                xmax = int(bndbox.find('xmax').text)
                ymax = int(bndbox.find('ymax').text)

                # YOLO形式に変換
                x_center = (xmin + xmax) / 2 / img_width
                y_center = (ymin + ymax) / 2 / img_height
                width = (xmax - xmin) / img_width
                height = (ymax - ymin) / img_height

                # 書き込み
                f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

        print(f"Converted {xml_file} to {output_txt_path}")

# パスの設定
xml_dir = os.path.join(PATH_TO_FCDetection, 'data')  # XMLファイルのディレクトリ
output_dir = os.path.join(PATH_TO_FCDetection, 'data_yolo')  # YOLO形式アノテーションの出力先
image_dir = os.path.join(PATH_TO_FCDetection, 'data')  # 画像が格納されているディレクトリ

# 実行
convert_xml_to_yolo(xml_dir, output_dir, image_dir)

Converted flowchart-example001.xml to /content/drive/MyDrive/programs/flow-chart-detection/data_yolo/flowchart-example001.txt
Converted flowchart-example002.xml to /content/drive/MyDrive/programs/flow-chart-detection/data_yolo/flowchart-example002.txt
Converted flowchart-example003.xml to /content/drive/MyDrive/programs/flow-chart-detection/data_yolo/flowchart-example003.txt
Converted flowchart-example004.xml to /content/drive/MyDrive/programs/flow-chart-detection/data_yolo/flowchart-example004.txt
Converted flowchart-example007.xml to /content/drive/MyDrive/programs/flow-chart-detection/data_yolo/flowchart-example007.txt
Converted flowchart-example008.xml to /content/drive/MyDrive/programs/flow-chart-detection/data_yolo/flowchart-example008.txt
Converted flowchart-example009.xml to /content/drive/MyDrive/programs/flow-chart-detection/data_yolo/flowchart-example009.txt
Converted flowchart-example013.xml to /content/drive/MyDrive/programs/flow-chart-detection/data_yolo/flowchart-example

In [ ]:
!ls

assets	 damo	   LICENSE  README_cn.md  requirements.txt  setup.py
configs  datasets  NOTICE   README.md	  scripts	    tools


In [ ]:
!git clone git@github.com:Megvii-BaseDetection/YOLOX.git
!ls
%cd YOLOX
!pip3 install -v -e .  # or  python3 setup.py develop

Cloning into 'YOLOX'...
Host key verification failed.
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.
drive  sample_data
[Errno 2] No such file or directory: 'YOLOX'
/content
Using pip 24.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
Obtaining file:///content
ERROR: file:///content does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [ ]:
!git clone https://github.com/tinyvision/damo-yolo.git
!ls damo-yolo/
%cd damo-yolo/
!pip install -r requirements.txt
import os
os.environ['PYTHONPATH'] = f"{os.getcwd()}:{os.environ.get('PYTHONPATH', '')}"

In [9]:
!ls

assets	 damo	   LICENSE  README_cn.md  requirements.txt  setup.py
configs  datasets  NOTICE   README.md	  scripts	    tools


In [10]:
import os
# %cd damo-yolo/
os.environ['PYTHONPATH'] = f"{os.getcwd()}:{os.environ.get('PYTHONPATH', '')}"
!python tools/demo.py --help


usage: DAMO-YOLO Demo [-h] [-f CONFIG_FILE] [-p PATH] [--camid CAMID] [--engine ENGINE]
                      [--device DEVICE] [--output_dir OUTPUT_DIR] [--conf CONF]
                      [--infer_size INFER_SIZE [INFER_SIZE ...]] [--end2end]
                      [--save_result SAVE_RESULT]
                      input_type

positional arguments:
  input_type            input type, support [image, video, camera]

options:
  -h, --help            show this help message and exit
  -f CONFIG_FILE, --config_file CONFIG_FILE
                        pls input your config file
  -p PATH, --path PATH  path to image or video
  --camid CAMID         camera id, necessary when input_type is camera
  --engine ENGINE       engine for inference
  --device DEVICE       device used to inference
  --output_dir OUTPUT_DIR
                        where to save inference results
  --conf CONF           conf of visualization
  --infer_size INFER_SIZE [INFER_SIZE ...]
                        test img size


In [ ]:
# !pip install cython
import sys
sys.path.append('/content/drive/MyDrive/programs/flow-chart-detection/damo-yolo')
print("sys.path ", sys.path)
# %cd damo-yolo/
!export PYTHONPATH="$PYTHONPATH:/content/drive/MyDrive/programs/flow-chart-detection/damo-yolo"
!echo $PYTHONPATH
!ls
!pip install git+https://github.com/cocodataset/cocoapi.git #subdirectory=PythonAPI # for Linux


sys.path  ['/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/usr/local/lib/python3.10/dist-packages/setuptools/_vendor', '/root/.ipython', '/tmp/tmpid1l89rn', '/content/drive/MyDrive/programs/flow-chart-detection/damo-yolo', '/content/drive/MyDrive/programs/flow-chart-detection/damo-yolo', '/content/drive/MyDrive/programs/flow-chart-detection/damo-yolo', '/content/drive/MyDrive/programs/flow-chart-detection/damo-yolo', '/content/drive/MyDrive/programs/flow-chart-detection/damo-yolo']
/env/python
assets	 damo	   LICENSE  README_cn.md  requirements.txt  setup.py
configs  datasets  NOTICE   README.md	  scripts	    tools
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-49nuzyih
  Running command git clone --filter=blob:none --quiet https://github.com/cocodataset

In [31]:
%cd ../
!cp -r damo-yolo /content/drive/MyDrive/programs/flow-chart-detection/


/content/damo-yolo


In [40]:
!pwd

/content/drive/MyDrive/programs/flow-chart-detection/damo-yolo


In [39]:
%cd /content/drive/MyDrive/programs/flow-chart-detection/damo-yolo
import sys
!pwd
!ls
sys.path.append('/content/drive/MyDrive/programs/flow-chart-detection/damo-yolo')
# !export PYTHONPATH="/env/python"
import os
os.environ['PYTHONPATH'] = "/env/python:/content/drive/MyDrive/programs/flow-chart-detection/damo-yolo"
# os.environ['PYTHONPATH'] = f"{os.getcwd()}:{os.environ.get('PYTHONPATH', '')}
# !export PYTHONPATH="$PYTHONPATH:/content/drive/MyDrive/programs/flow-chart-detection/damo-yolo"
# os.environ['PYTHONPATH'] = f"{os.getcwd()}:{os.environ.get('PYTHONPATH', '')}
!echo $PYTHONPATH
!python tools/demo.py image -f ./configs/damoyolo_tinynasL25_S.py --engine /content/drive/MyDrive/programs/flow-chart-detection/ckpt/damoyolo_tinynasL25_S_477.pth --conf 0.6 --infer_size 640 640 --device cuda --path ./assets/dog.jpg


/content/drive/MyDrive/programs/flow-chart-detection/damo-yolo
/content/drive/MyDrive/programs/flow-chart-detection/damo-yolo
assets	 damo	   demo     NOTICE	  README.md	    scripts   tools
configs  datasets  LICENSE  README_cn.md  requirements.txt  setup.py
/env/python:/content/drive/MyDrive/programs/flow-chart-detection/damo-yolo
Inference with torch engine!
/content/drive/MyDrive/programs/flow-chart-detection/damo-yolo/tools/demo.py:75: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded 

## pascal voc 形式の独自データを coco 形式に変える

In [41]:
import os
import json
import xml.etree.ElementTree as ET

# 入力ディレクトリ（XMLファイルと画像が格納されているディレクトリ）
input_dir = "../data/"
output_dir = "../data_coco_format"
os.makedirs(os.path.join(output_dir, "annotations"), exist_ok=True)
os.makedirs(os.path.join(output_dir, "images"), exist_ok=True)

# COCO形式のJSONファイル
output_json_path = os.path.join(output_dir, "annotations", "instances_custom.json")

# カテゴリの定義
"""
text
arrow
terminator
data
process
decision
connection
"""

categories = [
    {"id": 1, "name": "text", "supercategory": "object"},
    {"id": 2, "name": "arrow", "supercategory": "object"},
    {"id": 3, "name": "terminator", "supercategory": "object"},
    {"id": 4, "name": "data", "supercategory": "object"},
    {"id": 5, "name": "process", "supercategory": "object"},
    {"id": 6, "name": "decision", "supercategory": "object"},
    {"id": 7, "name": "connection", "supercategory": "object"},
    # 必要に応じてカテゴリを追加
]

category_name_to_id = {category["name"]: category["id"] for category in categories}

# COCO JSONの初期化
coco_format = {
    "images": [],
    "annotations": [],
    "categories": categories,
}

# アノテーションIDカウンタ
annotation_id = 1

# XMLファイルを処理
for xml_file in sorted(os.listdir(input_dir)):
    if not xml_file.endswith(".xml"):
        continue

    # XMLを解析
    tree = ET.parse(os.path.join(input_dir, xml_file))
    root = tree.getroot()

    # 画像情報
    filename = root.find("filename").text
    width = int(root.find("size/width").text)
    height = int(root.find("size/height").text)
    image_id = len(coco_format["images"]) + 1

    # COCO形式の画像情報を追加
    coco_format["images"].append({
        "id": image_id,
        "file_name": filename,
        "width": width,
        "height": height,
    })

    # 画像を出力フォルダにコピー（オプション）
    image_path = os.path.join(input_dir, filename)
    if os.path.exists(image_path):
        os.system(f"cp {image_path} {os.path.join(output_dir, 'images', filename)}")

    # アノテーション情報を追加
    for obj in root.findall("object"):
        category_name = obj.find("name").text
        if category_name not in category_name_to_id:
            continue  # 定義されていないカテゴリはスキップ

        category_id = category_name_to_id[category_name]
        bbox_xml = obj.find("bndbox")
        xmin = int(bbox_xml.find("xmin").text)
        ymin = int(bbox_xml.find("ymin").text)
        xmax = int(bbox_xml.find("xmax").text)
        ymax = int(bbox_xml.find("ymax").text)

        # COCO形式のbboxは[x, y, width, height]
        bbox = [xmin, ymin, xmax - xmin, ymax - ymin]
        area = bbox[2] * bbox[3]  # 面積

        coco_format["annotations"].append({
            "id": annotation_id,
            "image_id": image_id,
            "category_id": category_id,
            "bbox": bbox,
            "area": area,
            "segmentation": [],  # セグメンテーションは省略
            "iscrowd": 0,
        })
        annotation_id += 1

# JSONファイルとして保存
with open(output_json_path, "w") as json_file:
    json.dump(coco_format, json_file, indent=4)

print(f"COCO形式のJSONファイルが生成されました: {output_json_path}")


COCO形式のJSONファイルが生成されました: ../data_coco_format/annotations/instances_custom.json


In [42]:
!ln -s ../data_coco_format /content/drive/MyDrive/programs/flow-chart-detection/damo-yolo/datasets/custom_coco

In [43]:
!python tools/train.py -f configs/damoyolo_tinynasL25_S.py -d 1 -b 8 --fp16 -o --cache


usage: Damo-Yolo train parser [-h] [-f CONFIG_FILE] [--local_rank LOCAL_RANK]
                              [--tea_config TEA_CONFIG] [--tea_ckpt TEA_CKPT]
                              ...
Damo-Yolo train parser: error: unrecognized arguments: -d


# tmp




# end

In [ ]:
# COCO datasetで事前学習
from ultralytics import YOLO

# Load a model
# model = YOLO("yolo11n.pt")
model = YOLO("yolo11l.pt")

# Train the model
train_results = model.train(
    data="coco8.yaml",  # path to dataset YAML
    epochs=100,  # number of training epochs
    imgsz=640,  # training image size
    device="cpu",  # device to run on, i.e. device=0 or device=0,1,2,3 or device=cpu
)

# Evaluate model performance on the validation set
metrics = model.val()

# Perform object detection on an image
results = model(os.path.join(PATH_TO_FCDetection, 'images/200730_am01.jpg'))
results[0].show()

# Export the model to ONNX format
path = model.export(format="onnx")  # return path to exported model
# path = model.export(format="pth")  # return path to exported model

## yolo datasetに向けた事前準備

In [ ]:
# dataディレクトリから画像を data_yolo へコピー
import shutil

img_files = glob.glob(os.path.join(PATH_TO_FCDetection, 'data', '*'))
for path1 in img_files:
  if path1.rsplit('.', 1)[1] == 'xml' or path1.rsplit('.', 1)[1] == 'txt':
    continue
    print(path1)
  shutil.copy(path1, os.path.join(PATH_TO_FCDetection, 'data_yolo/images/train'))

KeyboardInterrupt: 

In [ ]:
# yoloの学習
from ultralytics import YOLO
model = YOLO(os.path.join(PATH_TO_FCDetection, 'runs_l/train/weights/best.pt'))
model.train(
    data=os.path.join(PATH_TO_FCDetection, 'data_yolo/yolov11_l_241212.yaml'),  # 絶対パス
    epochs=100,
    imgsz=640
)

In [ ]:
# 保存モデルの確認
!ls -la /content/drive/MyDrive/programs/flow-chart-detection/
!ls -la runs/detect/train/weights
# !mv yolo11n.pt yolo11_l_2412120234.pt
# !cp runs/detect/train/weights/best.pt /content/drive/MyDrive/programs/flow-chart-detection/runs_l/train/weights/yolo11_l_24120239.pt

total 28
drwx------ 2 root root 4096 Nov 29 01:18 ckpt
drwx------ 2 root root 4096 Nov 26 19:54 data
drwx------ 4 root root 4096 Dec 12 04:29 data_yolo
drwx------ 2 root root 4096 Dec 11 03:22 images
drwx------ 2 root root 4096 Nov 26 22:51 output
drwx------ 4 root root 4096 Dec 11 04:18 runs_l
drwx------ 6 root root 4096 Dec 11 04:01 runs_n
ls: cannot access 'runs/detect/train/weights': No such file or directory


In [ ]:
# 保存したパラメータによる推論
import os
from ultralytics import YOLO
import glob
# Load a model
# model = YOLO("yolo11n.pt")
model = YOLO(os.path.join(PATH_TO_FCDetection, 'runs_l/train/weights/yolo11_l_24120239.pt'))
# model = YOLO("yolo11l.pt

# Train the model
# train_results = model.train(
#     data="coco8.yaml",  # path to dataset YAML
#     epochs=100,  # number of training epochs
#     imgsz=640,  # training image size
#     device="cpu",  # device to run on, i.e. device=0 or device=0,1,2,3 or device=cpu
# )

# Evaluate model performance on the validation set
# metrics = model.val()

# results = model(os.path.join(PATH_TO_FCDetection, 'images/200730_am01.jpg'))
# results = model(os.path.join(PATH_TO_FCDetection, 'images/animals.jpeg'))
img_paths = glob.glob(os.path.join(PATH_TO_FCDetection, 'data_yolo/images/val', '*'))
for img_path in img_paths:
  results = model(img_path)
  results[0].show()
  print("type(results[0]) ", type(results[0]))
# results = model.predict(os.path.join(PATH_TO_FCDetection, 'data_yolo/images/animals.jpeg'))





In [ ]:
# 保存したぱらめーたによる推論
from ultralytics import YOLO

# Load a model
model = YOLO(os.path.join(PATH_TO_FCDetection, 'runs_l/train/weights/best.pt'))

results = model(os.path.join(PATH_TO_FCDetection, 'images/200730_am01.jpg'))
# results[0].show()
print("type(results[0]) ", type(results[0]))
print("len(results) ", len(results))
for box in results[0].boxes:
    print("type(box) ", type(box))
    print("type(box.xyxy) ", type(box.xyxy))
    x1, y1, x2, y2 = box.xyxy[0].tolist()  # バウンディングボックスの座標 (左上と右下)
    confidence = box.conf[0].item()        # 信頼度
    cls = int(box.cls[0].item())          # クラスID
    print(f"Class: {cls}, Confidence: {confidence:.2f}, BBox: [{x1:.2f}, {y1:.2f}, {x2:.2f}, {y2:.2f}]")